# Imports

In [1]:
import requests
from bs4 import BeautifulSoup 
import re
import pandas as pd 
import os 
import time
import datetime
import csv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pandas.io.json import json_normalize
import csv
from pyquery import PyQuery as pq
from lxml import etree
import requests

# Michigan State

## Article webscrape

### The State News

Articles containing "gun control" from Feb 17, 2018 to present.

Pulling the article links from the search page

In [2]:
## Request page
response = requests.get('http://statenews.com/search/?a=1&s=gun+control&ti=&ts_month=02&ts_day=17&ts_year=2018&te_month=06&te_day=14&te_year=2018&au=&tg=&ty=0&o=date')

## See the html for the whole page
soupresults1 = BeautifulSoup(response.text,'lxml')


In [3]:
## #Extract the links
StateNews = []

articles = soupresults1.find_all(class_='clearfix')

for a in articles:
    for link in a.find_all('a'):
        link = (link.get('href'))
        StateNews.append('http://statenews.com/' + link)
        

StateNews = set(StateNews)

Getting the text from articles

In [206]:
# response = BeautifulSoup(requests.get('http://statenews.com//article/2018/04/republican-gubernatorial-townhall').text,'xml')

In [207]:
# response = BeautifulSoup(requests.get('http://statenews.com//article/2018/02/bernie-sanders-visits-lansing-speaks-against-gop-tax-bill').text,'xml')

# for ptag in response.find_all(class_='article-copy drop-cap'):
#  for p in ptag.find_all('p'):
#         print(p.text)


In [4]:
StateNews

{'http://statenews.com//article/2018/02/bernie-sanders-visits-lansing-speaks-against-gop-tax-bill',
 'http://statenews.com//article/2018/02/rally-for-common-sense-gun-violence-prevention-to-be-held-thursday',
 'http://statenews.com//article/2018/02/video-bernie-sanders-lansing-gop-tax-bill',
 'http://statenews.com//article/2018/03/enough-is-enough-rally-at-capitol',
 'http://statenews.com//article/2018/03/msu-admissions-wont-penalize-prospective-and-admitted-students-for-peacefully-protesting',
 'http://statenews.com//article/2018/03/msu-students-community-react-to-central-michigan-shooting',
 'http://statenews.com//article/2018/03/national-school-walkout-planned-wednesday',
 'http://statenews.com//article/2018/03/parkland-students-at-msu-feature-story',
 'http://statenews.com//article/2018/03/students-parents-lawmakers-demand-gun-reform-on-capitol-steps',
 'http://statenews.com//article/2018/04/abdul-el-sayeds-talks-gun-control-immigration-and-more-at-rally',
 'http://statenews.com//a

In [5]:
StateNewsStories = []

for article in StateNews:
    fullarticle = []
    
    print(article)
    
    response = BeautifulSoup(requests.get(article).text,'xml')
    
    for ptag in response.find_all(class_='article-copy drop-cap'):
        for p in ptag.find_all('p'):
            fullarticle.append(p.text)
    
    articleinfo = {'Title': response.find_all(class_='article-headline')[0].text , 'Date':  response.find_all(class_='author-dateline')[0].text , 'Article': fullarticle}
        
    StateNewsStories.append(articleinfo)
    time.sleep(2)

http://statenews.com//article/2018/03/students-parents-lawmakers-demand-gun-reform-on-capitol-steps
http://statenews.com//article/2018/04/elissa-slotkin-running-for-congress
http://statenews.com//article/2018/04/abdul-el-sayeds-talks-gun-control-immigration-and-more-at-rally
http://statenews.com//article/2018/04/msu-criminology-professor-running-for-u-s-house-of-reps
http://statenews.com//article/2018/03/national-school-walkout-planned-wednesday
http://statenews.com//article/2018/03/parkland-students-at-msu-feature-story
http://statenews.com//article/2018/03/msu-students-community-react-to-central-michigan-shooting
http://statenews.com//article/2018/02/video-bernie-sanders-lansing-gop-tax-bill
http://statenews.com//article/2018/02/rally-for-common-sense-gun-violence-prevention-to-be-held-thursday
http://statenews.com//article/2018/02/bernie-sanders-visits-lansing-speaks-against-gop-tax-bill
http://statenews.com//article/2018/03/enough-is-enough-rally-at-capitol
http://statenews.com//ar

In [6]:
StateNewsStories

[{'Article': ['Students rallied on the steps of the Michigan Capitol building for gun reform Wednesday. Carrying signs and lifting their voices, they spoke about current events, like the Parkland school shooting, to demand change.',
   'Students hold up signs at the Students Demand Action on Gun Reform Rally at the Lansing state capitol. @thesnews pic.twitter.com/aRL0Az9ro6',
   'The rally was in part organized by 11th grade student Nash Salami from Plymouth High School with the help of Phoebe Hopps, founder of Women’s March Michigan.',
   '“I feel like it’s a really good way to raise awareness and bring attention to the issue of the things that are happening in our schools and spread the idea that our lives matter," Salami said. “We depend on our state legislators to pass laws that protect us in school.”',
   'Salami said it’s time for those who may not be of voting age to still make their voices heard.',
   '“For decades, adults have been debating this issue of gun control," Salami s

In [7]:
MSUStateNews = pd.DataFrame(StateNewsStories)

In [8]:
MSUStateNews

,Article,Date,Title
0,[Students rallied on the steps of the Michigan...,"March 15, 2018 9:55 am","Students, parents, lawmakers demand gun reform..."
1,[After working for the CIA and in the Office o...,"April 12, 2018 10:10 am",'Midwestern Democrat' Elissa Slotkin running f...
2,[Gubernatorial candidate Abdul El-Sayed stoppe...,"April 5, 2018 9:43 am","Abdul El-Sayed talks gun control, immigration ..."
3,[The 2016 presidential election made Chris Smi...,"April 26, 2018 10:24 am",MSU professor Chris Smith running for U.S. Hou...
4,"[Students, faculty and parents in East Lansing...","March 13, 2018 12:34 pm",National School Walkout planned for Wednesday ...
5,"[For some individuals of MSU, a shooting which...","March 1, 2018 10:09 am",MSU students from Parkland reflect after high ...
6,[Two people are dead following a shooting Frid...,"March 2, 2018 1:17 pm","MSU students, community react to Central Michi..."
7,[Thousands of people went to see Sen. Bernie S...,"February 26, 2018 2:52 pm","WATCH: Sen. Bernie Sanders visits Lansing, spe..."
8,[A Rally for Common Sense Gun Violence Prevent...,"February 22, 2018 9:34 am",'Rally for Common Sense Gun Violence Preventio...
9,"[According to Not One More Penny, the GOP tax ...","February 26, 2018 10:47 am","Sen. Bernie Sanders visits Lansing, speaks aga..."


#### Sentiment Analysis

from textblob import TextBlob

In [166]:
subjectivity = []
polarity = []

for word in range(len(MSUStateNews['Article'])):
    w = str(MSUStateNews["Article"][word])
    d = list(TextBlob(w).sentiment)
    polarity.append(d[0])
    subjectivity.append(d[1])

#### Is clustering interesting?

In [170]:
from __future__ import print_function
# gensim
from gensim import corpora, models, similarities, matutils
# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
# logging for gensim (set to INFO)
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.feature_extraction.text import CountVectorizer 

In [178]:
AllArticles = MSUStateNews['Article'].tolist()

In [180]:
AllArticles[0]

['MSU Admissions will not penalize prospective or admitted students for having a disciplinary or criminal record for peacefully protesting, according to a statement released on MSU Admissions’ Twitter account.',
 'In light of the recent protests against gun violence and other issues, a recent statement from James Cotter, the executive director of admissions and recruitment at MSU, said peaceful protests will not count against prospective and admitted students at MSU.',
 'To our future Spartans: pic.twitter.com/I5rZ7FNjDZ',
 '“Within the context of recent national discussions, ‘peaceful protest,’ a principal on which our nation was built, does not constitute as a threat,” the statement reads. “We treasure the freedom of speech, which is guaranteed by the guiding principles of the U.S. Constitution.”',
 'Cotter said in the wake of the Florida school shooting, The National Association for College Admission Counseling has had a great deal of discussion surrounding peaceful protests and how

In [183]:
vectorizer = CountVectorizer(ngram_range=(1,1), stop_words="english")

# call `fit` to build the vocabulary
vectorizer.fit(AllArticles[0])

x = vectorizer.fit_transform(AllArticles[0])
# # then, use `get_feature_names` to return the tokens
# print(vectorizer.get_feature_names())

counts = vectorizer.transform(AllArticles[0]).transpose()

In [193]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components= 5, max_iter = 15, random_state=0, 
                                n_jobs=-1, learning_method='batch')

In [194]:
topics = lda.fit_transform(x)

In [195]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [196]:
pyLDAvis.sklearn.prepare(lda, x, vectorizer)

/Users/vicky/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      32.368128        1       1  0.127063 -0.033532
1      21.844530        1       2 -0.093225 -0.032463
4      16.776911        1       3 -0.028433 -0.083530
3      16.746557        1       4 -0.016695  0.101081
0      12.263874        1       5  0.011290  0.048444, topic_info=     Category      Freq          Term     Total  loglift  logprob
term                                                             
137   Default  5.000000           msu  5.000000  30.0000  30.0000
8     Default  3.000000    admissions  3.000000  29.0000  29.0000
147   Default  4.000000      peaceful  4.000000  28.0000  28.0000
117   Default  2.000000      involved  2.000000  27.0000  27.0000
167   Default  4.000000      protests  4.000000  26.0000  26.0000
209   Default  1.000000       twitter  1.000000  25.0000  25.0000
7     Default  3.000000     admission  3.000000  24.0000  24.0000
185   Default  2.000000        school  2.000000  23.0000  23.0000
177   Default  2.000000     regarding  2.000000  22.0000  22.0000
104   Default  2.000000          high  2.000000  21.0000  21.0000
100   Default  1.000000           gun  1.000000  20.0000  20.0000
131   Default  3.000000         march  3.000000  19.0000  19.0000
174   Default  2.000000        recent  2.000000  18.0000  18.0000
112   Default  1.000000    individual  1.000000  17.0000  17.0000
44    Default  1.000000     community  1.000000  16.0000  16.0000
157   Default  1.000000          pose  1.000000  15.0000  15.0000
65    Default  1.000000  disciplinary  1.000000  14.0000  14.0000
92    Default  1.000000        future  1.000000  13.0000  13.0000
133   Default  1.000000         media  1.000000  12.0000  12.0000
135   Default  1.000000           mit  1.000000  11.0000  11.0000
30    Default  1.000000        center  1.000000  10.0000  10.0000
3     Default  2.000000     according  2.000000   9.0000   9.0000
168   Default  1.000000     questions  1.000000   8.0000   8.0000
56    Default  3.000000      criminal  3.000000   7.0000   7.0000
9     Default  2.000000      admitted  2.000000   6.0000   6.0000
195   Default  2.000000     statement  2.000000   5.0000   5.0000
204   Default  2.000000        threat  2.000000   4.0000   4.0000
164   Default  2.000000   prospective  2.000000   3.0000   3.0000
200   Default  2.000000   suspensions  2.000000   2.0000   2.0000
50    Default  6.000000        cotter  6.000000   1.0000   1.0000
...       ...       ...           ...       ...      ...      ...
4      Topic5  0.588608       account  1.080036   1.4915  -4.2721
178    Topic5  0.588608      released  1.080036   1.4915  -4.2721
149    Topic5  0.588608      penalize  1.080036   1.4915  -4.2721
148    Topic5  0.588608    peacefully  1.080036   1.4915  -4.2721
129    Topic5  0.588608         looks  1.080037   1.4915  -4.2721
62     Topic5  0.588608       decides  1.080037   1.4915  -4.2721
145    Topic5  0.588608        office  1.080037   1.4915  -4.2721
107    Topic5  0.588602    i5rz7fnjdz  1.080038   1.4915  -4.2721
43     Topic5  0.588602           com  1.080038   1.4915  -4.2721
193    Topic5  0.588602      spartans  1.080038   1.4915  -4.2721
153    Topic5  0.588602           pic  1.080038   1.4915  -4.2721
8      Topic5  1.570041    admissions  3.194430   1.3882  -3.2910
65     Topic5  0.588715  disciplinary  1.643808   1.0717  -4.2719
92     Topic5  0.588395        future  1.643856   1.0711  -4.2725
157    Topic5  0.588741          pose  1.646736   1.0699  -4.2719
112    Topic5  0.588741    individual  1.646736   1.0699  -4.2719
44     Topic5  0.588741     community  1.646736   1.0699  -4.2719
168    Topic5  0.588840     questions  1.706197   1.0346  -4.2717
175    Topic5  0.589057        record  1.780236   0.9925  -4.2714
182    Topic5  0.589006     responses  1.780258   0.9924  -4.2714
33     Topic5  0.588877       charges  1.780313   0.9922  -4.2717
137    Topic5  

### Spartan News Room

Articles containing "gun congrol" from Feb 13, 2018 to present.

http://news.jrn.msu.edu/?s=gun+control
pg 1-4

Pulling the article links from the search page

In [9]:
## Request page
response = requests.get('http://news.jrn.msu.edu/?s=gun+control')

## See the html for the whole page
soupresults2 = BeautifulSoup(response.text,'lxml')

soupresults2

<!DOCTYPE html>
<!--[if lt IE 7]> <html lang="en-US" class="no-js ie6"> <![endif]--><!--[if IE 7]>    <html lang="en-US" class="no-js ie7"> <![endif]--><!--[if IE 8]>    <html lang="en-US" class="no-js ie8"> <![endif]--><!--[if IE 9]>    <html lang="en-US" class="no-js ie9"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0" name="viewport"/>
<title>
		 gun control | Search Results  | Spartan Newsroom	</title>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<link href="http://news.jrn.msu.edu/xmlrpc.php" rel="pingback"/>
<title>Search Results for “gun control” – Spartan Newsroom</title>
<script>
			function whichHeader() {
				var screenWidth = document.documentElement.clientWidth,
				header_img;
				if (screenWidth <= 767) {
					header_img = 'http://news.jrn.msu.edu/wp-con

Getting the text from articles

In [10]:
SpartanNews = ['http://news.jrn.msu.edu/2018/06/gun-violence-laws-may-not-be-as-helpful-as-we-think/',
              'http://news.jrn.msu.edu/2018/05/stop-school-violence-act-receives-mostly-approval-with-some-concerns-if-it-goes-far-enough/',
              'http://news.jrn.msu.edu/2018/04/lansing-community-seeks-initiatives-to-promote-school-safety/',
              'http://news.jrn.msu.edu/2018/04/east-lansing-school-district-tightens-security/',
              'http://news.jrn.msu.edu/2018/04/okemos-schools-to-install-buzzer-and-lock-system/',
              'http://news.jrn.msu.edu/2018/04/michael-lynn-kappash-richards-use-platform-to-protest-injustice/',
              'http://news.jrn.msu.edu/2018/04/lansing-residents-react-to-citys-march-for-our-lives/',
              'http://news.jrn.msu.edu/2018/03/students-at-grand-ledge-high-school-join-nationwide-walkouts/',
              'http://news.jrn.msu.edu/2018/03/national-school-walkout-draws-out-ingham-students/',
              'http://news.jrn.msu.edu/2018/03/national-school-walkout-draws-out-ingham-students/student-led-walkout/',
              'http://news.jrn.msu.edu/2018/03/east-lansing-students-plan-march-14-gun-control-walkout/',
              'http://news.jrn.msu.edu/2018/04/nra-influence-stretches-beyond-campaign-contributions/']

In [11]:
SpartanNewsStories = []

for article in SpartanNews:
    fullarticle2 = []
    
    print(article)
    
    response = BeautifulSoup(requests.get(article).text,'lxml')
    
    for ptag in response.find_all(class_='entry-content clearfix'):
        for p in ptag.find_all('p'):
            fullarticle2.append(p.text)
            
    articleinfo2 = {'Title': response.find_all(class_='entry-title')[0].text , 'Date':  response.find_all(class_='entry-date updated dtstamp pubdate')[0].text , 'Article': fullarticle2}
        
    SpartanNewsStories.append(articleinfo2)
    time.sleep(2)


http://news.jrn.msu.edu/2018/06/gun-violence-laws-may-not-be-as-helpful-as-we-think/
http://news.jrn.msu.edu/2018/05/stop-school-violence-act-receives-mostly-approval-with-some-concerns-if-it-goes-far-enough/
http://news.jrn.msu.edu/2018/04/lansing-community-seeks-initiatives-to-promote-school-safety/
http://news.jrn.msu.edu/2018/04/east-lansing-school-district-tightens-security/
http://news.jrn.msu.edu/2018/04/okemos-schools-to-install-buzzer-and-lock-system/
http://news.jrn.msu.edu/2018/04/michael-lynn-kappash-richards-use-platform-to-protest-injustice/
http://news.jrn.msu.edu/2018/04/lansing-residents-react-to-citys-march-for-our-lives/
http://news.jrn.msu.edu/2018/03/students-at-grand-ledge-high-school-join-nationwide-walkouts/
http://news.jrn.msu.edu/2018/03/national-school-walkout-draws-out-ingham-students/
http://news.jrn.msu.edu/2018/03/national-school-walkout-draws-out-ingham-students/student-led-walkout/


IndexError: list index out of range

In [290]:
SpartanNewsStories

[{'Article': ['On May 8, United States Representatives Debbie Dingell and Fred Upton of Michigan proposed a bill that would grant access to funds for states to enforce laws that enable police to confiscate guns from gun owners who are deemed dangerous.',
   'The debate about gun ownership and gun control in the United States has been a large one in recent times, especially with amount of gun violence and mass shootings there have been in the last few years, and some agree that the new proposed bill would aid in slowing down the crime rates.',
   '“I think it puts some responsibility back into the hands of the gun owner,” says Richard White, a Michigan police officer.',
   'White says that gun owners can be dangerous for a multitude of reasons. As well as not being mentally stable or responsible enough for handling a gun on their own, many gun owners do not practice measures to ensure that their firearms are in a safe place inaccessible to others.',
   '“Many crimes involving guns aren’

In [291]:
MSUSpartanNews = pd.DataFrame(SpartanNewsStories)

In [292]:
MSUSpartanNews

,Article,Date,Title
0,"[On May 8, United States Representatives Debbi...","June 9, 2018",Gun violence laws may not be as helpful as we ...
1,"[The Students, Teachers, and Officers Preventi...","May 7, 2018",STOP School Violence Act receives mostly appro...
2,[The Lansing Community is implementing new way...,"April 30, 2018",Lansing community seeks initiatives to promote...
3,[Less than a month after students participated...,"April 12, 2018",East Lansing schools tighten security
4,[The Okemos board of education voted to lock a...,"April 11, 2018",Okemos schools to install buzzer and lock system
5,"[Michael Lynn, the former quarterback of Lansi...","April 2, 2018","Michael Lynn, Kappash Richards use platform to..."
6,[Thousands of people marched from the Hall of ...,"April 2, 2018",Lansing residents react to city’s ‘March for O...
7,"[On March 14, 2018, roughly 200 Grand Ledge Hi...","March 28, 2018",Students at Grand Ledge High School join natio...
8,[Ingham County students joined thousands from ...,"March 14, 2018",National School Walkout draws out Ingham Count...


In [ ]:
# StateNewsStories = []

# for article in StateNews:
#     fullarticle = []
    
#     print(article)
    
#     response = BeautifulSoup(requests.get(article).text,'xml')
    
#     for ptag in response.find_all(class_='article-copy drop-cap'):
#         for p in ptag.find_all('p'):
#             fullarticle.append(p.text)
    
#     articleinfo = {'Title': response.find_all(class_='article-headline')[0].text , 'Date':  response.find_all(class_='author-dateline')[0].text , 'Article': fullarticle}
        
#     StateNewsStories.append(articleinfo)
#     time.sleep(2)

## Marshall University 

In [220]:
article = ['https://marshallparthenon.com/16806/news/activists-take-to-streets-at-march-for-our-lives-in-huntington/']

In [235]:
response = BeautifulSoup(requests.get('https://marshallparthenon.com/16806/news/activists-take-to-streets-at-march-for-our-lives-in-huntington/'))

## See the html for the whole page
soupresults1 = BeautifulSoup(response.text,'xml')

/Users/vicky/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/vicky/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


TypeError: object of type 'Response' has no len()

In [229]:
soupresults1

<?xml version="1.0" encoding="utf-8"?>

In [242]:
response = BeautifulSoup(requests.get('http://www.herald-dispatch.com/wcn/wc_opinion/gun-education-the-way-to-real-gun-control/article_bf23d96f-0209-5f12-9bd7-0c755a008e48.html').text,'xml')
    

/Users/vicky/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:132: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(data)


In [243]:
response

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html>
<html lang="en">
<head>
<base href="http://www.herald-dispatch.com/content/tncms/live/"/>
<meta content="addcomment" name="keywords"/>
<meta content="addcomment" name="news_keywords"/>
<meta content="&quot;I don't want gun-totting students on campus&quot;, Anthony Jenkins West Virginia State said.&quot;Do something even if it is wrong,&quot; Dave Peyton, recently wrote in a column in The Herald Dispatch" name="description"/>
<meta content="" name="author"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1, user-scalable=0, shrink-to-fit=no" name="viewport"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="black-translucent" name="apple-mobile-web-app-status-bar-style"/>
<meta content="article" property="og:type"/>
<meta content="http://www.herald-dispatch.com/wcn/wc_opinion/gun-education-the-way-to-real-gun-control/article_bf23d96f-0209-5f12-9bd7-0c755a008e48.html" property="og:url"/>
<

## University of New Hampshire

In [245]:
## Request page
please = requests.get('http://tnhdigital.com/college-republicans-democrats-talk-policy-and-issues-in-student-debate-p2322-218.htm')

## See the html for the whole page
soupresults1 = BeautifulSoup(please.text,'lxml')

/Users/vicky/anaconda3/lib/python3.6/site-packages/bs4/builder/_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  self.parser.feed(markup)


In [246]:
soupresults1

<html><body><p>ï»¿

</p>
<title>The New Hampshire</title>
<meta content="The New Hampshire" name="keywords"/>
<meta content="Put your description here..." name="description"/>
<link href="/clients/tnhdigital/favicon2.ico" rel="shortcut icon"/>
<script language="javascript" src="/bl.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/jquery.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/Gallery/jquery.ad-gallery.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/Watermark/jquery.watermarkinput.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/CSSMessageBox/js/jquery.ui.draggable.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/CSSMessageBox/js/jquery.alerts.js" type="text/javascript"></script>
<script language="javascript" src="/JQuery/calendar/jquery.datepick.js" type="text/javascript"></script>
<link href="/JQuery/calendar/jquery.datepi